今天讲讲 Union-Find 算法，也就是常说的并查集算法，主要是解决图论中「动态连通性」问题的。

Given the vertices and edges between them, how could we quickly check whether two vertices are connected? For example, Figure 5 shows the edges between vertices, so how can we efficiently check if 0 is connected to 3, 1 is connected to 5, or 7 is connected to 8? We can do so by using the “disjoint set” data structure, also known as the “union-find” data structure. Note that others might refer to it as an algorithm. In this Explore Card, the term “disjoint set” refers to a data structure.

The primary use of disjoint sets is to address the connectivity between the components of a network. The “network“ here can be a computer network or a social network. For instance, we can use a disjoint set to determine if two people share a common ancestor.

# 问题介绍
简单说，动态连通性其实可以抽象成给一幅图连线。比如下面这幅图，总共有 10 个节点，他们互不相连，分别用 0~9 标记：
![](images/uf_1.jfif)

现在我们的 Union-Find 算法主要需要实现这两个 API

In [ ]:
class UF:
    def __init__(self, n):
        pass
    
    # 将 p 和 q 连接
    def union(self, p, q):
        pass
    
    # 判断 p 和 q 是否连通
    def connected(self, p, q):
        pass
    
    # 返回图中有多少个连通分量
    def count(self):
        pass

这里所说的「连通」是一种等价关系，也就是说具有如下三个性质：

1. 自反性：节点p和p是连通的。
2. 对称性：如果节点p和q连通，那么q和p也连通。
3. 传递性：如果节点p和q连通，q和r连通，那么p和r也连通。

比如说之前那幅图，0～9 任意两个不同的点都不连通，调用connected都会返回 false，连通分量为 10 个。

如果现在调用union(0, 1)，那么 0 和 1 被连通，连通分量降为 9 个。

再调用union(1, 2)，这时 0,1,2 都被连通，调用connected(0, 2)也会返回 true，连通分量变为 8 个。
![](images/uf_2.jfif)
判断这种「等价关系」非常实用，比如说编译器判断同一个变量的不同引用，比如社交网络中的朋友圈计算等等。

这样，你应该大概明白什么是动态连通性了，Union-Find 算法的关键就在于union和connected函数的效率。那么用什么模型来表示这幅图的连通状态呢？用什么数据结构来实现代码呢？

注意我刚才把「模型」和具体的「数据结构」分开说，这么做是有原因的。
1. 因为我们使用森林（若干棵树）来表示图的动态连通性，用数组来具体实现这个森林。
2. 我们设定树的每个节点有一个指针指向其父节点，如果是根节点的话，这个指针指向自己。

比如说刚才那幅 10 个节点的图，一开始的时候没有相互连通，就是这样：
![](images/uf_3.jfif)

In [2]:
class UF():
    def __init__(self, n):
        self.count = n
        # 父节点指针初始指向自己
        self.parent = [i for i in range(n)]
    
    # 返回图中有多少个连通分量
    def count(self):
        return self.count
    
    # 将 p 和 q 连接
    def union(self, p, q):
        """
        如果某两个节点被连通，则让其中的（任意）
        一个节点的根节点接到另一个节点的根节点上
        """
        rootP = self.find(p)
        rootQ = self.find(q)
        
        if rootP == rootQ:
            return
        # 将两棵树合并为一棵
        self.parent[rootP] = rootQ
        
        # parent[rootQ] = rootP 也一样
        self.count -= 1
        
    
    # 返回某个节点 x 的根节点
    def find(self, x):
        # 根节点的 parent[x] == x
        while self.parent[x] != x:
            x = self.parent[x]
        return x
    
    # 判断 p 和 q 是否连通
    def connected(self, p, q):
        """
        这样，如果节点p和q连通的话，它们一定拥有相同的根节点
        """
        rootP = self.find(p)
        rootQ = self.find(q)
        
        return rootP == rootQ

例如我们把q和q链接在一起， 如下图所示
![](images/uf_4.jfif)

判断p和q是否连通：
![](images/uf_5.jfif)

至此，Union-Find 算法就基本完成了。那么这个算法的复杂度是多少呢？
1. 我们发现，主要 APIconnected和union中的复杂度都是find函数造成的，所以说它们的复杂度和find一样。
2. find主要功能就是从某个节点向上遍历到树根，其时间复杂度就是树的高度。
3. 我们可能习惯性地认为树的高度就是logN，但这并不一定。logN的高度只存在于平衡二叉树，对于一般的树可能出现极端不平衡的情况，使得「树」几乎退化成「链表」，树的高度最坏情况下可能变成N。

![](images/uf_6.jfif)

所以说上面这种解法，find,union,connected的时间复杂度都是 **O(N)**。这个复杂度很不理想的，你想图论解决的都是诸如社交网络这样数据规模巨大的问题，对于union和connected的调用非常频繁，每次调用需要线性时间完全不可忍受。

问题的关键在于，如何想办法避免树的不平衡呢？
1. 我们要知道哪种情况下可能出现不平衡现象，关键在于union过程
2. 我们一开始就是简单粗暴的把p所在的树接到q所在的树的根节点下面，那么这里就可能出现「头重脚轻」的不平衡状况，比如下面这种局面：

![](images/uf_7.jfif)

长此以往，树可能生长得很不平衡。我们其实是希望，小一些的树接到大一些的树下面，这样就能避免头重脚轻，更平衡一些。解决方法是额外使用一个size数组，记录每棵树包含的节点数，我们不妨称为「重量」：

# 平衡性优化

我们一开始就是简单粗暴的把p所在的树接到q所在的树的根节点下面，那么这里就可能出现「头重脚轻」的不平衡状况，比如下面这种局面：


长此以往，树可能生长得很不平衡。我们其实是希望，小一些的树接到大一些的树下面，这样就能避免头重脚轻，更平衡一些。解决方法是额外使用一个size数组，记录每棵树包含的节点数，我们不妨称为「重量」：

这样，通过比较树的重量，就可以保证树的生长相对平衡，树的高度大致在logN这个数量级，极大提升执行效率。

此时，find,union,connected的时间复杂度都下降为 **O(logN)**，即便数据规模上亿，所需时间也非常少。


In [3]:
class UFWithWeight():
    def __init__(self, n):
        self.count = n
        # 父节点指针初始指向自己
        self.parent = [i for i in range(n)]
        # 新增一个数组记录树的“重量”
        self.size = [1] * n
    
    def count(self):
        return self.count
    
    def union(self, p, q):
        rootP = self.find(p)
        rootQ = self.find(q)
        
        if rootP == rootQ:
            return
        
        # 小树接到大树下面，较平衡
        if self.size[rootP] > self.size(rootQ):
            self.parent[rootQ] = rootP
            self.size[rootP] += self.size[rootQ]
        else:
            self.parent[rootP] = rootQ
            self.size[rootQ] += self.size[rootP]
        
        self.count -= 1
        
    
    # 返回某个节点 x 的根节点
    def find(self, x):
        while self.parent[x] != x:
            x = self.parent[x]
        return x
        
    def connected(self, p, q):
        rootP = self.find(p)
        rootQ = self.find(q)
        
        return rootP == rootQ

# 路径压缩

这步优化特别简单，所以非常巧妙。我们能不能进一步压缩每棵树的高度，使树高始终保持为常数？ 这样find就能以 O(1) 的时间找到某一节点的根节点，相应的，connected和union复杂度都下降为 O(1)。

要做到这一点，非常简单，只需要在find中加一行代码：
```
parent[x] = parent[parent[x]]
```

![](images/UF_10.gif)

这个操作有点匪夷所思，看个 GIF 就明白它的作用了（为清晰起见，这棵树比较极端). 可见，调用find函数每次向树根遍历的同时，顺手将树高缩短了，最终所有树高都不会超过 3（union的时候树高可能达到 3）。


In [4]:
class UFWithWeight_Compressed():
    def __init__(self, n):
        self.count = n
        # 父节点指针初始指向自己
        self.parent = [i for i in range(n)]
        # 新增一个数组记录树的“重量”
        self.size = [1] * n
    
    def count(self):
        return self.count
    
    def union(self, p, q):
        rootP = self.find(p)
        rootQ = self.find(q)
        
        if rootP == rootQ:
            return
        
        # 小树接到大树下面，较平衡
        if self.size[rootP] >= self.size(rootQ):
            self.parent[rootQ] = rootP
            self.size[rootP] += self.size[rootQ]
        else:
            self.parent[rootP] = rootQ
            self.size[rootQ] += self.size[rootP]
        
        self.count -= 1
        
    
    # 返回某个节点 x 的根节点
    def find(self, x):
        while self.parent[x] != x:
            # 进行路径压缩
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return x
        
    def connected(self, p, q):
        rootP = self.find(p)
        rootQ = self.find(q)
        
        return rootP == rootQ

可见，调用find函数每次向树根遍历的同时，顺手将树高缩短了，最终所有树高都不会超过 3（union的时候树高可能达到 3）。

PS：读者可能会问，这个 GIF 图的find过程完成之后，树高恰好等于 3 了，但是如果更高的树，压缩后高度依然会大于 3 呀？不能这么想。这个 GIF 的情景是我编出来方便大家理解路径压缩的，但是实际中，每次find都会进行路径压缩，所以树本来就不可能增长到这么高，你的这种担心应该是多余的。

# Union-Find 算法应用


## DFS 的替代方案

很多使用 DFS 深度优先算法解决的问题，也可以用 Union-Find 算法解决。

# 判定合法算式


# Reference 
1. [Union-Find 并查集算法详解](https://mp.weixin.qq.com/s/Czqc1zjfVQg6Pbtfnp_1Bg)
2. [Union-Find 算法怎么应用？](https://mp.weixin.qq.com/s/sPqFpQvHNbDe5NO2DYVIKQ)